[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_upT_toWSsxcWBuk8TKLLddQoogKaZWjlsI3YRcSGz-n8ZrMgBQrQII6BdTXbo7fSZUg1YhAZitlVJR3s9qgMZbOfqdil1u6h2YYidVwiQNdWMSWSjrMGDCh2cO9gM22oOrWczdDG5r5l_Uokj3eKAvMeBehYbldbG_1vkoHPkw?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_upT_toWSsxcWBuk8TKLLddQoogKaZWjlsI3YRcSGz-n8ZrMgBQrQII6BdTXbo7fSZUg1YhAZitlVJR3s9qgMZbOfqdil1u6h2YYidVwiQNdWMSWSjrMGDCh2cO9gM22oOrWczdDG5r5l_Uokj3eKAvMeBehYbldbG_1vkoHPkw)

In [28]:
%%capture
! pip install git+https://github.com/openai/swarm.git
! pip install pandas_ta

In [29]:
import yfinance as yf
from swarm import Swarm, Agent
import pandas_ta as ta
import pandas as pd
import os

In [30]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [20]:
# 股價走勢查詢函數
def get_stock_price_history(symbol, period = "1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return history.tail().to_dict('records')
    except:
        return f"無法獲取 {symbol} 的歷史價格"

In [31]:
get_stock_price_history('2330.TW')

[{'Open': 1100.0,
  'High': 1125.0,
  'Low': 1095.0,
  'Close': 1125.0,
  'Volume': 69289786,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Open': 1150.0,
  'High': 1160.0,
  'Low': 1130.0,
  'Close': 1130.0,
  'Volume': 48758688,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Open': 1110.0,
  'High': 1130.0,
  'Low': 1105.0,
  'Close': 1105.0,
  'Volume': 45211419,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Open': 1100.0,
  'High': 1115.0,
  'Low': 1100.0,
  'Close': 1100.0,
  'Volume': 27263215,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Open': 1100.0,
  'High': 1110.0,
  'Low': 1100.0,
  'Close': 1100.0,
  'Volume': 22021522,
  'Dividends': 0.0,
  'Stock Splits': 0.0}]

In [32]:
# 股票資訊查詢函數
def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        return {
            '公司名稱': info.get('longName', 'N/A'),
            '現價': info.get('currentPrice', 'N/A'),
            '市值': info.get('marketCap', 'N/A'),
            '產業': info.get('industry', 'N/A'),
            '52週高點': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52週低點': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except:
        return f"無法獲取 {symbol} 的資訊"

In [33]:
get_stock_info('2330.TW')

{'公司名稱': 'Taiwan Semiconductor Manufacturing Company Limited',
 '現價': 1100.0,
 '市值': 28525969014784,
 '產業': 'Semiconductors',
 '52週高點': 1160.0,
 '52週低點': 578.0}

In [22]:
# 基本財務數據查詢函數
def get_financial_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        # 獲取最近的財務報表
        financials = stock.financials.iloc[:, 0]
        balance_sheet = stock.balance_sheet.iloc[:, 0]
        return {
            '營收': financials.get('Total Revenue', 'N/A'),
            '淨利': financials.get('Net Income', 'N/A'),
            '總資產': balance_sheet.get('Total Assets', 'N/A'),
            '總負債': balance_sheet.get('Total Liabilities Net Minority Interest', 'N/A'),
            'ROE': stock.info.get('returnOnEquity', 'N/A'),
            '本益比': stock.info.get('trailingPE', 'N/A')
        }
    except:
        return f"無法獲取 {symbol} 的財務數據"

In [34]:
get_financial_data('2330.TW')

{'營收': 2161735800000.0,
 '淨利': 851740000000.0,
 '總資產': 5532196600000.0,
 '總負債': 2078330100000.0,
 'ROE': 0.28027,
 '本益比': 27.173914}

In [36]:
# 技術分析函數
def get_technical_analysis(symbol, period= "6mo"):
    try:
        # 獲取歷史數據
        stock = yf.Ticker(symbol)
        df = stock.history(period=period)

        # 計算技術指標
        # RSI
        df['RSI'] = ta.rsi(df['Close'], length=14)

        # MACD
        macd = ta.macd(df['Close'])
        df = pd.concat([df, macd], axis=1)

        # 移動平均線
        df['SMA_20'] = ta.sma(df['Close'], length=20)
        df['SMA_60'] = ta.sma(df['Close'], length=60)

        # KD指標
        stoch = ta.stoch(df['High'], df['Low'], df['Close'])
        df = pd.concat([df, stoch], axis=1)

        # 取最新的技術指標數據
        latest = df.iloc[-1]

        analysis_result = {
            'RSI': round(latest['RSI'], 2),
            'MACD': round(latest['MACD_12_26_9'], 2),
            'MACD_Signal': round(latest['MACDs_12_26_9'], 2),
            'SMA_20': round(latest['SMA_20'], 2),
            'SMA_60': round(latest['SMA_60'], 2),
            'K值': round(latest['STOCHk_14_3_3'], 2),
            'D值': round(latest['STOCHd_14_3_3'], 2),
            '收盤價': round(latest['Close'], 2)
        }

        # 產生技術分析信號
        signals = generate_trading_signals(analysis_result)

        return {
            'indicators': analysis_result,
            'signals': signals,
            'historical_data': df.tail().to_dict('records')
        }

    except Exception as e:
        return f"無法進行技術分析: {str(e)}"

In [37]:
def generate_trading_signals(data):
    signals = []

    # RSI 分析
    if data['RSI'] > 70:
        signals.append("RSI 顯示超買情況")
    elif data['RSI'] < 30:
        signals.append("RSI 顯示超賣情況")

    # MACD 分析
    if data['MACD'] > data['MACD_Signal']:
        signals.append("MACD 形成黃金交叉")
    elif data['MACD'] < data['MACD_Signal']:
        signals.append("MACD 形成死亡交叉")

    # KD 分析
    if data['K值'] < 20 and data['D值'] < 20:
        signals.append("KD 指標處於超賣區")
    elif data['K值'] > 80 and data['D值'] > 80:
        signals.append("KD 指標處於超買區")

    # 均線分析
    if data['SMA_20'] > data['SMA_60']:
        signals.append("20日均線突破60日均線")
    elif data['SMA_20'] < data['SMA_60']:
        signals.append("20日均線跌破60日均線")

    return signals

In [38]:
get_technical_analysis('2330.TW')

{'indicators': {'RSI': 56.1,
  'MACD': 14.54,
  'MACD_Signal': 13.49,
  'SMA_20': 1085.0,
  'SMA_60': 1057.08,
  'K值': 50.29,
  'D值': 62.98,
  '收盤價': 1100.0},
 'signals': ['MACD 形成黃金交叉', '20日均線突破60日均線'],
 'historical_data': [{'Open': 1100.0,
   'High': 1125.0,
   'Low': 1095.0,
   'Close': 1125.0,
   'Volume': 69289786,
   'Dividends': 0.0,
   'Stock Splits': 0.0,
   'RSI': 64.3740713118774,
   'MACD_12_26_9': 12.704426740210693,
   'MACDh_12_26_9': 1.5452761611420396,
   'MACDs_12_26_9': 11.159150579068653,
   'SMA_20': 1074.8904296875,
   'SMA_60': 1052.9775329589843,
   'STOCHk_14_3_3': 72.22222222222223,
   'STOCHd_14_3_3': 67.5925925925926},
  {'Open': 1150.0,
   'High': 1160.0,
   'Low': 1130.0,
   'Close': 1130.0,
   'Volume': 48758688,
   'Dividends': 0.0,
   'Stock Splits': 0.0,
   'RSI': 65.26270143723433,
   'MACD_12_26_9': 15.568774775829525,
   'MACDh_12_26_9': 3.5276993574086966,
   'MACDs_12_26_9': 12.041075418420828,
   'SMA_20': 1077.846173095703,
   'SMA_60': 1054.460

In [39]:
# 總管理 Agent 的指令
general_agent = Agent(
    name="總管理Agent",
    instructions="""你是股票市場查詢的總管理者。
    根據使用者的問題類型，將請求轉發給適當的專家：
    - 股價相關問題 -> 股價查詢專家
    - 財務數據相關問題 -> 財務分析專家
    - 技術指標相關問題 -> 技術分析專家
    如果使用者想要完整分析，可以依序調用多個專家的服務。""",
    functions=[get_stock_info, get_financial_data, get_technical_analysis]
)

# 股價查詢專家 Agent
price_agent = Agent(
    name="股價查詢專家",
    instructions="""你是專門處理股價查詢的專家。
    使用 get_stock_info 和 get_stock_price_history 函數來回答：
    - 即時股價
    - 歷史走勢
    - 52週高低點等資訊
    請用專業且易懂的方式解釋數據。""",
    functions=[get_stock_info, get_stock_price_history]
)

# 創建財務分析專家 Agent
financial_agent = Agent(
    name="財務分析專家",
    instructions="""你是專門分析財務數據的專家。
    使用 get_financial_data 函數來分析：
    - 基本財務指標
    - 獲利能力
    - 財務結構
    提供專業的財務分析建議。""",
    functions=[get_financial_data]
)

# 技術分析專家 Agent
technical_agent = Agent(
    name="技術分析專家",
    instructions="""你是專門進行技術分析的專家。
    使用 get_technical_analysis 函數來分析：
    - RSI、MACD、KD等技術指標
    - 移動平均線
    - 產生交易信號
    請用專業且易懂的方式解釋技術分析結果，並提供可能的交易建議。
    記得提醒使用者技術分析僅供參考，需要綜合其他因素做決定。""",
    functions=[get_technical_analysis]
)

# 創建 Swarm client
client = Swarm()



In [42]:
print("歡迎使用股票市場助手！")
print("您可以詢問：")
print("1. 股票即時報價與走勢")
print("2. 基本財務分析")
print("3. 基本技術分析")
print("請輸入股票代號（例如：2330.TW）和您的問題。")

context_variables = {}
current_agent = general_agent

while True:
    user_input = input("\n您的問題: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("謝謝使用，再見！")
        break

    response = client.run(
        agent=current_agent,
        messages=[{"role": "user", "content": user_input}],
        context_variables=context_variables
    )

    print(f"\n{response.agent.name}:", response.messages[-1]["content"])
    current_agent = response.agent
    context_variables = response.context_variables


歡迎使用股票市場助手！
您可以詢問：
1. 股票即時報價與走勢
2. 基本財務分析
3. 基本技術分析
請輸入股票代號（例如：2330.TW）和您的問題。

您的問題: 請問2330.TW 最新股價

總管理Agent: 2330.TW（台積電）的最新股價是 1100 元。

您的問題: 請提供2330.TW目前股價和本益比

總管理Agent: 2330.TW (Taiwan Semiconductor Manufacturing Company Limited) 的目前股價為 1100 元。根據最新的財務數據，其本益比為 27.17。

您的問題: 請分析2330.TW和2308.TW的技術指標，從技術面來看，目前哪一個股票的進場時機更好？

總管理Agent: 根據技術指標分析：

**2330.TW**
- RSI：56.1，表示相對強弱指數在中性偏強的區域。
- MACD：14.54，MACD形成黃金交叉，表示短期內價格可能上漲。
- 20日均線突破60日均線，呈現上漲動能。

**2308.TW**
- RSI：51.97，位於中立區域。
- MACD：4.74，MACD形成死亡交叉，表示短期可能下降。
- 20日均線突破60日均線，意味著短期內價格正在試圖上漲，但目前動能不足。

總結來說，2330.TW 在 MACD 黃金交叉及重要均線突破方面的表現更強勢，技術面上進場時機相對較好。相比之下，2308.TW 的技術指標顯示較弱的動能。

您的問題: 如果現在要在2330.TW和2308.TW選一個買進，建議是？

總管理Agent: 根據獲取的資料，我將為2330.TW（台積電）和2308.TW（台達電）進行比較分析：

### 2330.TW 台積電
- **股價**：現價 1100.0
- **財務數據**：
  - 營收：2,161,735,800,000
  - 淨利：851,740,000,000
  - 總資產：5,532,196,600,000
  - 總負債：2,078,330,100,000
  - ROE：28.03%
  - 本益比：27.17
- **技術分析**：
  - MACD 黃金交叉
  - 20日均線突破60日均線
  - RSI：56.1
  - K值：50.29，D值：62.98

### 2308.TW 台達

KeyboardInterrupt: Interrupted by user